In [16]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

In [3]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [17]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=x)

In [18]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_3 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten_3 True
20 dense_7 True
21 batch_normalization_5 True
22 dropout_5 True
23 dense_8 True
24 batch_normalization_6 True
25 dropout_6 True
26 dense_9 True


In [19]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.2,
                                         rotation_range=15,
                                         height_shift_range=0.1,
                                         width_shift_range=0.1,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [21]:
vgg16_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    #optimizer=optimizers.Adam(lr=1e-3),
                    optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=5)])

Epoch 1/10
156/156 [==============================] - 344s 2s/step - loss: 1.1995 - acc: 0.6256 - val_loss: 1.1122 - val_acc: 0.6610
Epoch 2/10
156/156 [==============================] - 342s 2s/step - loss: 0.3809 - acc: 0.8757 - val_loss: 1.3641 - val_acc: 0.6315
Epoch 3/10
156/156 [==============================] - 343s 2s/step - loss: 0.2655 - acc: 0.9151 - val_loss: 1.0938 - val_acc: 0.6842
Epoch 4/10
156/156 [==============================] - 340s 2s/step - loss: 0.2022 - acc: 0.9348 - val_loss: 1.1601 - val_acc: 0.6886
Epoch 5/10
156/156 [==============================] - 339s 2s/step - loss: 0.1610 - acc: 0.9479 - val_loss: 1.3931 - val_acc: 0.6585
Epoch 6/10
156/156 [==============================] - 339s 2s/step - loss: 0.1454 - acc: 0.9544 - val_loss: 1.6431 - val_acc: 0.6089
Epoch 7/10
156/156 [==============================] - 336s 2s/step - loss: 0.1268 - acc: 0.9588 - val_loss: 1.8447 - val_acc: 0.5862
Epoch 8/10
156/156 [==============================] - 337s 2s/step - 

In [9]:
# vgg16_model.optimizer.lr = 1e-4

# vgg16_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=5,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size)

Epoch 1/5
156/156 [==============================] - 344s 2s/step - loss: 0.1321 - acc: 0.9568 - val_loss: 1.1900 - val_acc: 0.7036
Epoch 2/5
156/156 [==============================] - 344s 2s/step - loss: 0.1315 - acc: 0.9580 - val_loss: 1.5762 - val_acc: 0.6494
Epoch 3/5
156/156 [==============================] - 345s 2s/step - loss: 0.1116 - acc: 0.9632 - val_loss: 1.6493 - val_acc: 0.6071
Epoch 4/5
156/156 [==============================] - 341s 2s/step - loss: 0.1080 - acc: 0.9657 - val_loss: 1.7442 - val_acc: 0.6150
Epoch 5/5
156/156 [==============================] - 341s 2s/step - loss: 0.1055 - acc: 0.9660 - val_loss: 2.0907 - val_acc: 0.5675


In [15]:
for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),#,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])


vgg16_model.save('vgg16_fine_tuning_model.h5')

Epoch 1/20
156/156 [==============================] - 347s 2s/step - loss: 0.4006 - acc: 0.8888 - val_loss: 1.1897 - val_acc: 0.7010
Epoch 2/20
156/156 [==============================] - 344s 2s/step - loss: 0.1026 - acc: 0.9699 - val_loss: 1.1307 - val_acc: 0.7478
Epoch 3/20
156/156 [==============================] - 344s 2s/step - loss: 0.0882 - acc: 0.9734 - val_loss: 5.5791 - val_acc: 0.3584
Epoch 4/20
156/156 [==============================] - 343s 2s/step - loss: 0.0492 - acc: 0.9854 - val_loss: 0.6537 - val_acc: 0.8576
Epoch 5/20
156/156 [==============================] - 341s 2s/step - loss: 0.1739 - acc: 0.9506 - val_loss: 3.0969 - val_acc: 0.5431
Epoch 6/20
156/156 [==============================] - 340s 2s/step - loss: 0.0538 - acc: 0.9841 - val_loss: 1.5184 - val_acc: 0.6498
Epoch 7/20
156/156 [==============================] - 340s 2s/step - loss: 0.0339 - acc: 0.9890 - val_loss: 1.8684 - val_acc: 0.6533
Epoch 8/20
156/156 [==============================] - 340s 2s/step - 